# Smile Data Analysis Example

This notebook demonstrates how to use the `smiledata` library to analyze experiment data.

In [ ]:
from smiledata import load_json, load_latest
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns

# Set up nice plotting defaults
sns.set_theme(style="whitegrid", context="notebook")

## Load Data

In [ ]:
# Load the most recent data file from the data folder
# You can also use load_json("path/to/specific/file.json")
data = load_latest("../data")
print(f"Loaded {len(data)} participants")
data.summary()

## Filter to Complete Participants

In [ ]:
# Filter to only complete participants (consented, done, not withdrawn)
complete = data.complete_only()
print(f"Complete participants: {len(complete)}")

## Convert to DataFrame

In [ ]:
# Get trial-level DataFrame with all study data
trials = complete.to_trials_df()
trials.head(10)

## Basic Analysis

In [ ]:
# Calculate mean RT and accuracy per participant
if "rt" in trials.columns and "correct" in trials.columns:
    participant_stats = trials.group_by("participant_id").agg(
        pl.col("rt").mean().alias("mean_rt"),
        pl.col("correct").mean().alias("accuracy")
    )
    display(participant_stats)
else:
    print("RT and/or correct columns not found in trial data")
    display(complete.to_participants_df())

## Demographics

In [ ]:
# Get demographics DataFrame
demographics = complete.demographics_df()
demographics.head()

## Plotting

In [ ]:
from smiledata.plotting import plot_completion_rate, plot_trial_rt_distribution

# Plot completion rate
ax = plot_completion_rate(data)
plt.show()

In [ ]:
# Plot RT distribution (if RT column exists)
if "rt" in trials.columns:
    ax = plot_trial_rt_distribution(trials, rt_column="rt")
    plt.show()

## Individual Participant Access

In [ ]:
# Access individual participant
first_participant = complete[0]
print(f"ID: {first_participant.id}")
print(f"Conditions: {first_participant.conditions}")
print(f"Trial count: {first_participant.trial_count}")

In [ ]:
# Get participant's trial data as DataFrame
first_participant.study_data_to_polars()